In [96]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt

In [97]:
# Read data
data = pd.read_csv('RawData/Returns.csv', index_col="RIC")
data.columns = pd.to_datetime(data.columns, dayfirst=True)

In [98]:
# Check if all rows only contain numbers (or NaNs)
n = 0
for type in data.dtypes.values:
    if type.name != "float64":
        n += 1
if(n == 0):
    print(f"{n} rows have not the expected data type float64. Perfect!")
else:
    print(f"{n} rows dont have the expected data type float64. Check carefully!")

0 rows have not the expected data type float64. Perfect!


This section runs the filter proposed by Ince & Porter 2006. As this leaves many extreme returns in the dataframe, we just exlude returns > 100%. So dont run the next two cells.

In [99]:
# extremes = data[data[data.columns[1:]] > 300]
# extremes.dropna(axis = 0, how = 'all', inplace = True)
# extremes.dropna(axis = 1, how = 'all', inplace = True)
# print(f"The return data contains {extremes.count().sum()} entries where the monthly return was above 300 %. These data points could be errors.")

In [100]:
# # This cell iterates through all the extreme positiv (return > 300 %) and checks if there are extreme negative returns the month befor or after so that the gains are almost completely compensated. In that chase there was probably a wrong price reported for the stock in that month and therefore wrong returns have been calculated. These retruns are set to NaN.
# # The errors dataframe collected all the detected values as well as the return from the prio/next month. It was used for getting an overview over the data but is not necessary for further work.

# extreme_count = 0
# nan_count = 0
# cleaned_data = data.copy(deep=True)
# # errors = DataFrame.copy(extremes).drop(extremes.index)
# for ric in extremes.index:
#     for extreme_date in extremes.columns:
#         extreme_val = extremes.loc[ric, extreme_date]
#         if not np.isnan(extreme_val):
#             prior_date = data.columns[data.columns.get_loc(extreme_date)-1]
#             prior_val = data.loc[ric, prior_date]
#             prior_calc = ((1+extreme_val/100)*(1+prior_val/100)-1)*100

#             next_date = data.columns[data.columns.get_loc(extreme_date)+1]
#             next_val = data.loc[ric, next_date]
#             next_calc = ((1+extreme_val/100)*(1+next_val/100)-1)*100
#             if (prior_calc < 50 and next_calc < 50) or (np.isnan(prior_calc) and np.isnan(next_calc)):
#                 # errors.at[ric, prior_date] = prior_val
#                 # errors.at[ric, extreme_date] = extreme_val
#                 # errors.at[ric, next_date] = next_val
#                 cleaned_data.at[ric, prior_date] = np.nan
#                 cleaned_data.at[ric, extreme_date] = np.nan
#                 cleaned_data.at[ric, next_date] = np.nan
#                 # print(f"[{ric}]\nPrior: {prior_val}   {prior_date}\nExtreme: {extreme_val}   {extreme_date}\nNext: {next_val}   {next_date}")
#                 if np.isnan(prior_calc) and np.isnan(next_calc):
#                     nan_count += 1
#                 else:
#                     extreme_count +=1
#             elif prior_calc < 50 or np.isnan(prior_calc):
#                 # errors.at[ric, prior_date] = prior_val
#                 # errors.at[ric, extreme_date] = extreme_val
#                 cleaned_data.at[ric, prior_date] = np.nan
#                 cleaned_data.at[ric, extreme_date] = np.nan
#                 # print(f"[{ric}]\nPrior: {prior_val}   {prior_date}\nExtreme: {extreme_val}   {extreme_date}\nNext: {next_val}   {next_date}")
#                 if np.isnan(prior_calc):
#                     nan_count += 1
#                 else:
#                     extreme_count +=1
#             elif next_calc < 50 or np.isnan(next_calc):
#                 # errors.at[ric, extreme_date] = extreme_val
#                 # errors.at[ric, next_date] = next_val
#                 cleaned_data.at[ric, extreme_date] = np.nan
#                 cleaned_data.at[ric, next_date] = np.nan
#                 # print(f"[{ric}]\nPrior: {prior_val}   {prior_date}\nExtreme: {extreme_val}   {extreme_date}\nNext: {next_val}   {next_date}")
#                 if np.isnan(next_calc):
#                     nan_count += 1
#                 else:
#                     extreme_count +=1
            
# print(f"{nan_count} periods have been removed due to NaN values in the prior/next period. {extreme_count} periods have been removed due to extreme loss/gain that is compensated in the prior/next period. Both factors indicate errors.")

Run the next cell instead for a simpler but more effective removal of outliers.

In [101]:
# Remove returns that are bigger/smaller than p % of the positive/negative returns

from numpy import negative


p = 1

# Turn dataframe into one dimensional numpy array and remove nans
returns = data.to_numpy().flatten()
returns = returns[~np.isnan(returns)]
n_before = len(returns)
positives = returns[returns > 0]
negatives = returns[returns < 0]

# Calculate boundaries based one perentile
lower_boundary = np.percentile(negatives, p)
upper_boundary = np.percentile(positives, 100-p)
print(f"Number of non-nan returns before outlier removal: {n_before}")
print(f"{p} % of the positive returns are bigger than {upper_boundary}")
print(f"{p} % of the negative returns are smaller than {lower_boundary}")

# Go through columns and remove returns outside of interval (lower_boundary, upper_boundary)
cleaned_data = data.copy()
for column in data.columns:
    c = data[column]
    cleaned_data[column] = c[c.between(lower_boundary, upper_boundary)]

# Print number of non-nan returns after outlier removal
returns = cleaned_data.to_numpy().flatten()
returns = returns[~np.isnan(returns)]
n_after = len(returns)
print(f"Number of non-nan returns after outlier removal: {n_after}")
print(f"Difference: {n_before-n_after}")

Number of non-nan returns before outlier removal: 216331
1 % of the positive returns are bigger than 145.9016393
1 % of the negative returns are smaller than -60.0
Number of non-nan returns after outlier removal: 214334
Difference: 1997


In [102]:
# Save dataframe
cleaned_data.to_pickle("UsableData/Returns.pkl")